In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import webbrowser
from random import shuffle
import cv2

In [2]:
num_input = 10222
num_classes = 120 
learning_rate = 0.001
batch_size = 64
display_step = 20
dropout = 0.8

In [3]:
X = tf.placeholder(tf.float32, [None, None])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

In [4]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [5]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [6]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 100, 100, 3])

    
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    conv3 = maxpool2d(conv3, k=2)

    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
    conv4 = maxpool2d(conv4, k=2)

    print conv4
    print weights['wd1'].get_shape().as_list()[0]
    fc1 = tf.reshape(conv4, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    print(fc1)
    # Output, class prediction
    mat_mul = tf.matmul(fc1, weights['out'])
    print(mat_mul)
    out = tf.add(mat_mul, biases['out'])
    print(out)
    return out

In [7]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 4])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 4, 8])),
    'wc3': tf.Variable(tf.random_normal([5, 5, 8, 16])),
    'wc4': tf.Variable(tf.random_normal([5, 5, 16, 32])),

    'wd1': tf.Variable(tf.random_normal([7*7*32, 600])),
    'out': tf.Variable(tf.random_normal([600, num_classes]))
}

In [8]:
biases = {
    'bc1': tf.Variable(tf.random_normal([4])),
    'bc2': tf.Variable(tf.random_normal([8])),
    'bc3': tf.Variable(tf.random_normal([16])),
    'bc4': tf.Variable(tf.random_normal([32])),
    'bd1': tf.Variable(tf.random_normal([600])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [9]:
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

Tensor("MaxPool_3:0", shape=(?, 7, 7, 32), dtype=float32)
1568
Tensor("dropout/mul:0", shape=(?, 600), dtype=float32)
Tensor("MatMul_1:0", shape=(?, 120), dtype=float32)
Tensor("Add_1:0", shape=(?, 120), dtype=float32)


In [10]:
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [12]:
folder  = '/home/bhawani/Desktop/Practice/Capstone_problem/train'
images_list = os.listdir(folder)
shuffle(images_list)
train_images = images_list[:9500]
test_images = images_list[9500:]
labels_df = pd.read_csv('labels.csv')
labels_df = pd.concat([labels_df,pd.get_dummies(labels_df['breed'], prefix='breed')],axis=1)
labels_df = labels_df.drop(['breed'],axis=1)

In [13]:
def resized(img):
    return cv2.resize(img,(100,100))

In [14]:
def hsv(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [15]:
def next_batch(images_list,labels,step,batch_size):
    images_array = list()
    y_array = list()
    for i in range((step-1)*batch_size,(step)*batch_size):
        img = cv2.imread(folder+'/'+images_list[i])
        images_array.append(resized(hsv(img)).flatten())
        y = np.array(labels[labels.id == images_list[i][:-4]])
        y = np.delete(y, [0])
        y_array.append(y)
    return np.array(images_array).T,np.array(y_array)

In [16]:
tf.summary.scalar('accuracy',accuracy)
tf.summary.scalar('loss',loss_op)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('/home/bhawani/Desktop/Practice/Capstone_problem/checkpoints_3rd', tf.get_default_graph())

In [18]:
epochs=15
with tf.Session() as sess:
    sess.run(init)
    num_steps = int(len(train_images)/batch_size)
    print(num_steps)
    for e in range(epochs):
        for step in range(1, num_steps+1):
            batch_x, batch_y = next_batch(train_images,labels_df,step,batch_size)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
            if step % display_step == 0 or step == 1:
                loss, acc ,summary = sess.run([loss_op, accuracy,merged], feed_dict={X: batch_x,
                                                                     Y: batch_y,
                                                                     keep_prob: 1.0})
                
                print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
                save_path = saver.save(sess, "/home/bhawani/Desktop/Practice/Capstone_problem/checkpoints_3rd/model_"+str(e)+'_'+str(step)+".ckpt")
        
        test_array = list()
        y_array = list()
        for i in range(len(test_images)):
            img = cv2.imread(folder+'/'+test_images[i])
            test_array.append(resized(img).flatten())
            y = np.array(labels_df[labels_df.id == test_images[i][:-4]])
            y = np.delete(y,[0])
            y_array.append(y)
        print("Validation Accuracy of "+str(e)+':', sess.run(accuracy, feed_dict={X: np.array(test_array).T,Y: np.array(y_array),keep_prob: 1.0}))
    writer.add_summary(summary,step)
    print("Optimization Finished!")
writer.close()

148
Step 1, Minibatch Loss= 1147232640.000000, Training Accuracy= 0.03125
Step 20, Minibatch Loss= 394901376.000000, Training Accuracy= 0.00000
Step 40, Minibatch Loss= 285375296.000000, Training Accuracy= 0.00000
Step 60, Minibatch Loss= 215207744.000000, Training Accuracy= 0.04688
Step 80, Minibatch Loss= 211705584.000000, Training Accuracy= 0.00000
Step 100, Minibatch Loss= 159937120.000000, Training Accuracy= 0.00000
Step 120, Minibatch Loss= 164702624.000000, Training Accuracy= 0.00000
Step 140, Minibatch Loss= 138526576.000000, Training Accuracy= 0.00000
('Validation Accuracy of 0:', 0.005540166)
Step 1, Minibatch Loss= 126501088.000000, Training Accuracy= 0.01562
Step 20, Minibatch Loss= 100003776.000000, Training Accuracy= 0.00000
Step 40, Minibatch Loss= 81105136.000000, Training Accuracy= 0.00000
Step 60, Minibatch Loss= 76561680.000000, Training Accuracy= 0.00000
Step 80, Minibatch Loss= 67811248.000000, Training Accuracy= 0.00000
Step 100, Minibatch Loss= 53621528.000000, T

KeyboardInterrupt: 

In [19]:
with tf.Session() as sess:
    saver.restore(sess, "/home/bhawani/Desktop/Practice/Capstone_problem/checkpoints_3rd/model_12_140.ckpt")
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    test_array = list()
    y_array = list()
    for i in range(400):
            img = cv2.imread(folder+'/'+train_images[i])
            test_array.append(resized(img).flatten())
            y = np.array(labels_df[labels_df.id == train_images[i][:-4]])
            y = np.delete(y,[0])
            y_array.append(y)
    predictions = sess.run(prediction, feed_dict={X: np.array(test_array).T,Y: np.array(y_array),keep_prob: 1.0})
    print np.argmax(predictions, 1)
    print np.argmax(np.array(y_array),1)

INFO:tensorflow:Restoring parameters from /home/bhawani/Desktop/Practice/Capstone_problem/checkpoints_3rd/model_12_140.ckpt
[  0 115   0   0   0   0   0   0   0   0   0   0   0   0   0 111   0   0
   0   0   0   0   0   0   0   0   0   0   0 111   0   0   0   0   0  43
   0  10   0   0 109  10   0   0   0  10   0   0   0  32   0   0  46  18
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 110   0   0   0
   0   0   0   0   0   0   0   0 105   0   0   0  54   0   0   0   0   0
   0   0  54   0   0   0  56   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  43   0   0   0   0   0   0   0   0  42   0   0   0   0
   0   0   0   0  44   0   0   0   0   0   0   0   0   0   0   0  11   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 110   0   0   0
  69   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
